In [1]:
pip install simpy

Note: you may need to restart the kernel to use updated packages.


In [17]:
import simpy
import random
import statistics

In [42]:
maquinas_escaneo = 3
carril_autos = 2
carril_camiones = 1

tiempo_escaneo_auto = 5
tiempo_escaneo_camion = 18

tiempo_revision_auto = 15
tiempo_revision_camion = 25

espera_escaneo_auto = 2
espera_escaneo_camion = 5
espera_revision_auto = 10
espera_revision_camion = float("inf")

proporcion_llegada_auto = 0.80
proporcion_llegada_camion = 0.20


tiempo_simulacion = 240

seed = 77

llegada_minima = 2
llegada_maxima = 8

In [43]:
class CentroRevision:
    def __init__(self, env):
        self.env = env

        self.escaneo = simpy.Resource(env, maquinas_escaneo)
        self.revision_autos = simpy.Resource(env, capacity=carril_autos)
        self.revision_camiones = simpy.Resource(env, capacity=carril_camiones) #recursos

        self.autos_atendidos = 0
        self.camiones_atendidos = 0

        self.abandono_escaneo = {"auto": 0, "camion": 0}
        self.abandono_revision = {"auto": 0, "camion": 0}

    
    def atender_vehiculo(self, tipo):

        llegada = self.env.now
        limite = espera_escaneo_auto if tipo == "auto" else espera_escaneo_camion

        with self.escaneo.request() as escan:
            yield escan 

            espera = self.env.now - llegada
            if espera > limite:
                self.abandono_escaneo[tipo] += 1
                return  # cuando abandona antes de escaneo

            
            tiempo = tiempo_escaneo_auto if tipo == "auto" else tiempo_escaneo_camion
            yield self.env.timeout(tiempo) #cuanto se tarda en el escaneo dependiendo del vehivulo


        llegada_rev = self.env.now
        limite_rev = espera_revision_auto if tipo == "auto" else espera_revision_camion

        recurso = self.revision_autos if tipo == "auto" else self.revision_camiones

        with recurso.request() as recurs:
            yield recurs

            espera2 = self.env.now - llegada_rev
            if espera2 > limite_rev:
                self.abandono_revision[tipo] += 1
                return #aumenta el abandono en la revision

            
            dura = tiempo_revision_auto if tipo == "auto" else tiempo_revision_camion
            yield self.env.timeout(dura)

        
        if tipo == "auto":
            self.autos_atendidos += 1
        else:
            self.camiones_atendidos += 1 #se suman los vehicuklos revisados



    def llegada_vehiculos(self):
        i = 0
        while True:
            i += 1
            tipo = "camion" if random.random() < proporcion_llegada_camion else "auto"

            self.env.process(self.atender_vehiculo(tipo))

            tiemp = random.uniform(llegada_minima, llegada_maxima)
            yield self.env.timeout(tiemp)


        

In [44]:
def ejecutar_simulacion():
    env = simpy.Environment()
    cr = CentroRevision(env)

    env.process(cr.llegada_vehiculos())
    env.run(until=tiempo_simulacion)

    return cr

In [48]:
centroRevision = ejecutar_simulacion()

print("Autos atendidos:", centroRevision.autos_atendidos)
print("Camiones atendidos:", centroRevision.camiones_atendidos)

print("\nAbandonos antes del escaneo:", centroRevision.abandono_escaneo)
print("Abandonos antes de revisión:", centroRevision.abandono_revision)


Autos atendidos: 29
Camiones atendidos: 4

Abandonos antes del escaneo: {'auto': 0, 'camion': 0}
Abandonos antes de revisión: {'auto': 9, 'camion': 0}
